<a href="https://colab.research.google.com/github/newfull5/AI-Project/blob/master/nlp_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb transformers datasets torch

In [16]:
import argparse

def _get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument()
    return parser


args = argparse.Namespace(
  model_name="klue/bert-base",
  tokenizer_name="klue/bert-base",
  dataset_name="nsmc",
  save_dir_path="./",
  batch_size=4,
  max_epochs=3,
  patient=3,
  lr=3e-5
)

In [5]:
from datasets import load_dataset

dataset = load_dataset(args.dataset_name)

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset['test']['label'][25000:]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [30]:
from torch import nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class NsmcDataset(Dataset):
  def __init__(self, args, stage):
    super().__init__()
    self.stage = stage
    self.args = args
    self.documents, self.labels = self._get_data(args, stage)

  def _get_data(self, args, stage):
    dataset = load_dataset(args.dataset_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)

    if stage == 'train':
      documents = dataset['train']['document']
      labels = dataset['train']['label']
    elif stage == 'valid':
      documents = dataset['test']['document'][:25000]
      labels = dataset['test']['label'][:25000]
    elif stage == 'test':
      documents = dataset['test']['document'][25000:]
      labels = dataset['test']['label'][25000:]
    else:
      raise Exception("you can set stage only 'train', 'test' or 'valid'")

    return [self._preproc(tokenizer, text) for text in documents], labels

  def _preproc(self, tokenizer, text):
    tokenized = tokenizer(
      text=text,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
    )
    return tokenized
      
  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.documents[idx], self.labels[idx]

In [23]:
test_dataset = NsmcDataset(args, 'test')

  0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)